In [ ]:
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"YourProject\YourProject.gdb"

# --- Inputs ---
in_lines   = "Road_Centerlines"
endpoints  = "Road_Centerlin_FeatureVertic"     # start+end points per road centerline segment. File created using Geoprocessing (Intersect tool)

# --- Parameters ---
points_per_line = 40
buffer_dist_m   = 18                            # <-- minimum distance from (proxy) intersections

# --- Outputs ---
road_points_fc      = "RoadCenterPoints_N_per_line" 
endpoint_buf_fc     = "RoadEndpointBuffer18m"
road_points_clean   = "RoadCenterPoints_NoNearIntersections" 

# --- Checks ---
sr = arcpy.Describe(in_lines).spatialReference
print("Spatial Reference:", sr.name, sr.linearUnitName)
if (sr.linearUnitName or "").lower() not in ("meter", "metre"):
    raise ValueError(f"Input must be in meters. Current units: {sr.linearUnitName}")

# --- Prep: delete old outputs ---
for fc in [road_points_fc, endpoint_buf_fc, road_points_clean]:
    if arcpy.Exists(fc):
        arcpy.management.Delete(fc)

# --- 1) Build the buffer around endpoints ---
arcpy.analysis.Buffer(
    in_features=endpoints,
    out_feature_class=endpoint_buf_fc,
    buffer_distance_or_field=f"{buffer_dist_m} Meters",
    dissolve_option="ALL"
)

# --- 2) Create candidate points along each road line ---
arcpy.management.CreateFeatureclass(arcpy.env.workspace, road_points_fc, "POINT", spatial_reference=sr)

# Attributes to carry over
for fld_name, fld_type in [
    ("ST_NAME","TEXT"),
    ("Meters","DOUBLE"),
    ("Lanes","LONG"),
    ("DIR_TRAVEL","TEXT"),
    ("BRIDGE","TEXT"),
    ("SRC_OID","LONG"),
    ("X_POINT","DOUBLE"),
    ("Y_POINT","DOUBLE")
]:
    arcpy.management.AddField(road_points_fc, fld_name, fld_type)

with arcpy.da.SearchCursor(in_lines, ["OID@", "SHAPE@", "ST_NAME", "Meters", "Lanes", "DIR_TRAVEL", "BRIDGE"]) as sCur, \
     arcpy.da.InsertCursor(
         road_points_fc,
         ["SHAPE@", "ST_NAME", "Meters", "Lanes", "DIR_TRAVEL", "BRIDGE", "SRC_OID", "X_POINT", "Y_POINT"] # <---- you may chnage thing as per your road's centerline field attribute
     ) as iCur:

    for oid, geom, st_name, meters, lanes, dir_travel, bridge in sCur:
        if not geom or geom.length == 0:
            continue
        for k in range(1, points_per_line + 1):
            t = k / (points_per_line + 1.0)      # percent along
            pt = geom.positionAlongLine(t, True)  # point geometry
            x = pt.centroid.X
            y = pt.centroid.Y
            iCur.insertRow([pt, st_name, meters, lanes, dir_travel, bridge, oid, x, y])

print("Created candidate points:", road_points_fc)


# --- 3) Remove candidates near endpoints ---
# Make layers
arcpy.management.MakeFeatureLayer(road_points_fc, "cand_lyr")
arcpy.management.MakeFeatureLayer(endpoint_buf_fc, "buf_lyr")

# Select candidates that are within the no-stop zone
arcpy.management.SelectLayerByLocation(
    in_layer="cand_lyr",
    overlap_type="WITHIN_A_DISTANCE",
    select_features="buf_lyr",
    search_distance="0 Meters",   # buffer already contains the distance
    selection_type="NEW_SELECTION"
)

# Switch selection to keep only the ones OUTSIDE the buffer
arcpy.management.SelectLayerByAttribute("cand_lyr", "SWITCH_SELECTION")

# Export result
arcpy.management.CopyFeatures("cand_lyr", road_points_clean)

# Cleanup
arcpy.management.Delete("cand_lyr")
arcpy.management.Delete("buf_lyr")

print(f"Done: {road_points_clean} (all points ≥ {buffer_dist_m} m from endpoints).")

In [ ]:
HM_Pop_Lots   = "Pop_Landuse"    # <--- Filetered Lots based on the land uses (Residential, Office, Education, Mixed Use)

# --- Outputs ---
lot_centroids   = "Lots_Centroids"
lots_nearest    = "Lots_with_NearestRoad"          # centroids + NEAR_* + joined road attrs
nearest_pts_fc  = "Nearest_RoadPoints_From_Lots"   # points at the NEAR_X/NEAR_Y locations

# --- Sanity checks ---
for fc in (HM_Pop_Lots, road_points_fc):
    if not arcpy.Exists(fc):
        raise RuntimeError(f"Input not found: {fc}")

# --- 1) Centroids (inside each polygon) ---
if arcpy.Exists(lot_centroids):
    arcpy.management.Delete(lot_centroids)

# FeatureToPoint copies lot attributes (LOT_NO, POPULATION, AREA, LANDUSE, etc.)
arcpy.management.FeatureToPoint(HM_Pop_Lots, lot_centroids, "INSIDE")

# --- Make sure we’re in meters for NEAR distances ---
sr = arcpy.Describe(lot_centroids).spatialReference
if (sr.linearUnitName or "").lower() not in ("meter", "metre"):
    raise ValueError(f"Data should be projected in meters. Found: {sr.linearUnitName}")

# --- 2) Find nearest road point to each lot centroid ---
# Adds NEAR_FID, NEAR_DIST (meters), NEAR_X, NEAR_Y, etc. to lot_centroids
arcpy.analysis.Near(
    in_features=lot_centroids,
    near_features=road_points_fc,
    search_radius="",          # unlimited
    location="LOCATION",
    angle="ANGLE",
    method="PLANAR"
)

# Copy to a clean output FC (so we don’t keep modifying the source centroids)
if arcpy.Exists(lots_nearest):
    arcpy.management.Delete(lots_nearest)
arcpy.management.CopyFeatures(lot_centroids, lots_nearest)

# --- Join useful road-point attributes onto the centroid rows ---
# Adjust this list to whatever exists on your road points
candidate_fields = ["ST_NAME", "Lanes", "DIR_TRAVEL", "Meters", "SRC_OID", "SEQ", "LINE_PCT"]
existing_fields = {f.name for f in arcpy.ListFields(road_points_fc)}
fields_to_copy = [f for f in candidate_fields if f in existing_fields]

if fields_to_copy:
    try:
        arcpy.management.AddIndex(road_points_fc, "OBJECTID", "idx_objid")
    except Exception:
        pass

    arcpy.management.JoinField(
        in_data=lots_nearest, 
        in_field="NEAR_FID",
        join_table=road_points_fc,
        join_field="OBJECTID",
        fields=fields_to_copy
    )

print("✓ Centroids:", lot_centroids)
print("✓ Lots + nearest road point:", lots_nearest)
print("   Contains NEAR_DIST (meters), NEAR_X/NEAR_Y, and joined road attrs:", fields_to_copy)

# --- 3) (Optional) Create a point layer at the exact nearest road locations for mapping/labels ---
if arcpy.Exists(nearest_pts_fc):
    arcpy.management.Delete(nearest_pts_fc)

arcpy.management.XYTableToPoint(
    in_table=lots_nearest,
    out_feature_class=nearest_pts_fc,
    x_field="NEAR_X",
    y_field="NEAR_Y",
    coordinate_system=sr
)
print("✓ Nearest road locations FC:", nearest_pts_fc)

In [ ]:
arcpy.env.overwriteOutput = True

# --- Inputs ---
lots_nearest   = "Lots_with_NearestRoad"          # centroids with NEAR_X/NEAR_Y from Near tool

# --- Output ---
links_fc       = "LotToRoad_Lines"

# Sanity checks
if not arcpy.Exists(lots_nearest):
    raise RuntimeError(f"Input not found: {lots_nearest}")

# Spatial reference (use the same as centroids)
sr = arcpy.Describe(lots_nearest).spatialReference
if (sr.linearUnitName or "").lower() not in ("meter", "metre"):
    raise ValueError(f"Data should be projected in meters. Found: {sr.linearUnitName}")

# Create output feature class
if arcpy.Exists(links_fc):
    arcpy.management.Delete(links_fc)
arcpy.management.CreateFeatureclass(arcpy.env.workspace, links_fc, "POLYLINE", spatial_reference=sr)

# Add useful attributes (These are the field attributes used in this sample)
arcpy.management.AddField(links_fc, "LOT_NO", "TEXT", field_length=50)
arcpy.management.AddField(links_fc, "NEAR_DIST", "DOUBLE")
arcpy.management.AddField(links_fc, "NEAR_FID", "LONG")
arcpy.management.AddField(links_fc, "ST_NAME", "TEXT", field_length=100)
arcpy.management.AddField(links_fc, "LANES", "LONG")
arcpy.management.AddField(links_fc, "DIR_TRAVEL", "TEXT", field_length=20)

# Fields present on lots_nearest (adjust if your names differ)
in_fields = [
    "SHAPE@", 
    "LOT_NO",
    "NEAR_X", "NEAR_Y",
    "NEAR_DIST", "NEAR_FID",
    "ST_NAME", "Lanes", "DIR_TRAVEL"
]

# Only keep fields that actually exist
existing = {f.name for f in arcpy.ListFields(lots_nearest)}
in_fields = [f for f in in_fields if f == "SHAPE@" or f in existing]

out_fields = ["SHAPE@", "LOT_NO", "NEAR_DIST", "NEAR_FID", "ST_NAME", "LANES", "DIR_TRAVEL"]

# Reduce out_fields to match what's in in_fields
keep_map = {"LOT_NO","NEAR_DIST","NEAR_FID","ST_NAME","LANES","DIR_TRAVEL"}
out_fields = ["SHAPE@"] + [f for f in out_fields[1:] if f in keep_map and f in existing]

with arcpy.da.SearchCursor(lots_nearest, in_fields) as sCur, \
     arcpy.da.InsertCursor(links_fc, ["SHAPE@"] + out_fields[1:]) as iCur:

    # Build name->index for input row
    idx = {name:i for i,name in enumerate(in_fields)}

    for row in sCur:
        centroid_geom = row[idx["SHAPE@"]]
        # Skip rows where Near wasn't computed
        if "NEAR_X" not in idx or "NEAR_Y" not in idx:
            continue
        near_x = row[idx["NEAR_X"]]
        near_y = row[idx["NEAR_Y"]]
        if near_x in (None, "") or near_y in (None, ""):
            continue

        # Create PointGeometry for the near location
        near_pt = arcpy.PointGeometry(arcpy.Point(near_x, near_y), sr)

        # Build a two-vertex polyline: centroid -> nearest road point
        arr = arcpy.Array([centroid_geom.firstPoint, near_pt.firstPoint])
        line = arcpy.Polyline(arr, sr)

        # Gather attributes in the same order as out_fields (except SHAPE@ already handled)
        attrs = []
        for f in out_fields[1:]:
            src_name = f
            if f == "LANES" and "Lanes" in idx:
                src_name = "Lanes"
            if src_name in idx:
                attrs.append(row[idx[src_name]])
            else:
                attrs.append(None)

        iCur.insertRow([line] + attrs)

print("✓ Created link lines:", links_fc)